In [1]:
import os
import scipy.io
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from scipy.stats import ks_2samp
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor

from skopt import gp_minimize
from skopt.space import Real



In [2]:
# 获取当前目录
current_dir = './data'

file_list = []
# 遍历当前目录及其子目录|
for root, dirs, files in os.walk(current_dir):
    for file in files:
        if file.endswith(".mat"):
           file_list.append(file)

file_list.sort()

In [17]:
len(file_list)

114

In [4]:
L_fix = [0.03, 1]
D_fix = [4, 0.03]
R_fix = [4, 1]

L_change = [1, 1.3, 1.8, 2.3, 2.8, 3.3, 3.8, 4.3, 4.8]
D_change = [0.5, 0.8, 1.3, 1.8, 2.3]
R_change = [0.005, 0.02, 0.045, 0.07, 0.095]

angle = np.linspace(0, 180, 91)

In [5]:
L_param = np.array([[item] + L_fix for item in [item for item in L_change for _ in range(2)]])
D_param = np.array([D_fix + [item] for item in [item for item in D_change for _ in range(2)]])
R_param = np.array([[R_fix[0]]+[item]+[R_fix[1]] for item in [item for item in R_change for _ in range(2)]])

L_param = np.vstack([L_param, L_param, L_param])
D_param = np.vstack([D_param, D_param, D_param])
R_param = np.vstack([R_param, R_param, R_param])

print(L_param.shape, D_param.shape, R_param.shape)

(54, 3) (30, 3) (30, 3)


In [6]:
# 要复制的份数
k = len(angle)


# 初始化结果列表
L_arr = []

# 遍历原数组中的每一行
for row in L_param:
    # 遍历每一个要添加的元素
    for element in angle:
        # 将原行与新元素结合，并添加到结果中
        new_row = np.append(row, element)
        L_arr.append(new_row)

L_arr = np.array(L_arr)

# 初始化结果列表
D_arr = []

# 遍历原数组中的每一行
for row in D_param:
    # 遍历每一个要添加的元素
    for element in angle:
        # 将原行与新元素结合，并添加到结果中
        new_row = np.append(row, element)
        D_arr.append(new_row)

D_arr = np.array(D_arr)

# 初始化结果列表
R_arr = []

# 遍历原数组中的每一行
for row in R_param:
    # 遍历每一个要添加的元素
    for element in angle:
        # 将原行与新元素结合，并添加到结果中
        new_row = np.append(row, element)
        R_arr.append(new_row)

R_arr = np.array(R_arr)

print(L_arr.shape, D_arr.shape, R_arr.shape)

(4914, 4) (2730, 4) (2730, 4)


In [7]:
channel_list = ['C']*182 + ['S']*182 + ['X']*182
channel_list = channel_list * 19
len(channel_list)

10374

In [8]:
polar_list = ['hh']*91 + ['vv']*91 
polar_list = polar_list * 57
len(polar_list)

10374

In [9]:
encoder = OneHotEncoder()
encoded_cats = encoder.fit_transform(np.hstack([np.array([channel_list]).reshape(-1, 1), np.array([polar_list]).reshape(-1, 1)]))
encoded_cats = encoded_cats.toarray()

In [10]:
nonclass_feature = np.vstack([D_arr, L_arr, R_arr])
nonclass_feature.shape

(10374, 4)

In [11]:
nonclass_feature = np.vstack([D_arr, L_arr, R_arr])
nonclass_feature.shape

(10374, 4)

In [12]:
# y_list_max, y_list_min, y_list_50, y_list_20, y_list_80 = [], [], [], [], []

# for file in file_list:
#     data = scipy.io.loadmat(file)
#     max_data = data['fea_jing'][0][0][0][0][0][3].reshape(-1,)
#     min_data = data['fea_jing'][0][0][0][0][0][4].reshape(-1,)
#     percentile_50_data = data['fea_jing'][0][0][0][0][0][7].reshape(-1,)
#     percentile_20_data = data['fea_jing'][0][0][0][0][0][-4].reshape(-1,)
#     percentile_80_data = data['fea_jing'][0][0][0][0][0][-3].reshape(-1,)
#     y_list_max.append(max_data)
#     y_list_min.append(min_data)
#     y_list_50.append(percentile_50_data)
#     y_list_20.append(percentile_20_data)
#     y_list_80.append(percentile_80_data)

# total_y_max = np.array(y_list_max).reshape(114*91, 1)
# total_y_min = np.array(y_list_min).reshape(114*91, 1)
# total_y_50 = np.array(y_list_50).reshape(114*91, 1)
# total_y_20 = np.array(y_list_20).reshape(114*91, 1)
# total_y_80 = np.array(y_list_80).reshape(114*91, 1)

# total_y = np.hstack((total_y_max, total_y_min, total_y_50, total_y_20, total_y_80))

In [13]:
X_feature = np.hstack([nonclass_feature, encoded_cats])

X_feature = np.hstack([X_feature[:, :3], X_feature[:, 4:]])
# X_feature = X_feature[90::91, :]

In [14]:
# np.unique(X_feature[:, 3])

In [15]:
X_feature[:5]

array([[4.  , 0.03, 0.5 , 1.  , 0.  , 0.  , 1.  , 0.  ],
       [4.  , 0.03, 0.5 , 1.  , 0.  , 0.  , 1.  , 0.  ],
       [4.  , 0.03, 0.5 , 1.  , 0.  , 0.  , 1.  , 0.  ],
       [4.  , 0.03, 0.5 , 1.  , 0.  , 0.  , 1.  , 0.  ],
       [4.  , 0.03, 0.5 , 1.  , 0.  , 0.  , 1.  , 0.  ]])

In [20]:
y_list = []

for file in file_list:
    data = scipy.io.loadmat('./data/'+ file)
    mean = data['fea_jing'][0][0][0][0][0][1].reshape(-1,)
    y_list.append(mean)

total_y = np.array(y_list).reshape(114*91, 1)

In [21]:
y_feature = total_y

In [22]:
# y_feature = y_feature[90::91]

In [23]:
y_feature.shape

(10374, 1)

In [24]:
def pawn_sensitivity(X, Y, num_bins=500):
    sensitivities = []
    for i in range(X.shape[1]):  # 对每个特征进行 PAWN 灵敏度分析
        ks_statistics = []
        
        # 将特征 i 分成 num_bins 个区间
        percentiles = np.percentile(X[:, i], np.linspace(0, 100, num_bins + 1))
        
        for j in range(num_bins):
            # 获取区间范围
            mask = (X[:, i] >= percentiles[j]) & (X[:, i] < percentiles[j + 1])
            
            # 计算条件输出 Y 的分布
            Y_cond = Y[mask]
            
            # 计算条件分布 Y_cond 与整体分布 Y 的 KS 距离
            if len(Y_cond) > 0:
                ks_stat, _ = ks_2samp(Y, Y_cond)
                ks_statistics.append(ks_stat)
        
        # PAWN 敏感性指标为 KS 统计量的最大值
        sensitivities.append(np.max(ks_statistics))
    
    return sensitivities

In [25]:
# 计算每个特征的 PAWN 敏感性指标
sensitivities = pawn_sensitivity(X_feature, y_feature)

# 打印结果
for i, sensitivity in enumerate(sensitivities):
    print(f"Feature {i + 1} - PAWN Sensitivity: {sensitivity}")

Feature 1 - PAWN Sensitivity: 0.2534220165799113
Feature 2 - PAWN Sensitivity: 0.1256988625409678
Feature 3 - PAWN Sensitivity: 0.13774821669558507
Feature 4 - PAWN Sensitivity: 0.01971274339695389
Feature 5 - PAWN Sensitivity: 0.09856371698476968
Feature 6 - PAWN Sensitivity: 0.09393676498939657
Feature 7 - PAWN Sensitivity: 0.08550221708116446
Feature 8 - PAWN Sensitivity: 0.08550221708116446


In [21]:
# 计算每个特征与标签 y 的互信息
mi = mutual_info_regression(X_feature, y_feature)

# 打印结果
for i, score in enumerate(mi):
    print(f"Feature {i + 1} - Mutual Information with y: {score}")

/home/home_node7/cjz/miniconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Feature 1 - Mutual Information with y: 0.1221124016022328
Feature 2 - Mutual Information with y: 0.06573263637994908
Feature 3 - Mutual Information with y: 0.07331954139263486
Feature 4 - Mutual Information with y: 0.06158931116192834
Feature 5 - Mutual Information with y: 0.10646568025119252
Feature 6 - Mutual Information with y: 0.09928023052925883
Feature 7 - Mutual Information with y: 0.07616062834629767
Feature 8 - Mutual Information with y: 0.07616062834629767


In [22]:
spearman_corr = []
for i in range(X_feature.shape[1]):
    corr, _ = spearmanr(X_feature[:, i], y_feature)
    spearman_corr.append(corr)

# 打印结果
for i, corr in enumerate(spearman_corr):
    print(f"Feature {i + 1} - Spearman correlation with y: {corr}")

Feature 1 - Spearman correlation with y: 0.021654394834626986
Feature 2 - Spearman correlation with y: 0.04239313958128888
Feature 3 - Spearman correlation with y: 0.0470403427928678
Feature 4 - Spearman correlation with y: -0.008302441512802643
Feature 5 - Spearman correlation with y: 0.23142583719610613
Feature 6 - Spearman correlation with y: -0.2231233956833035
Feature 7 - Spearman correlation with y: 0.10240758223329717
Feature 8 - Spearman correlation with y: -0.10240758223329717


In [23]:
# 函数：计算互信息
def compute_mutual_info(x, y):
    return mutual_info_regression(x, y)

# 函数：计算特定区间内第5维特征的互信息敏感性
def mutual_info_in_interval(bounds, x, y, feature_idx=3):
    start, end = bounds
    mask = (x[:, feature_idx] >= start) & (x[:, feature_idx] < end)
    if mask.sum() > 1:  # 确保至少有两个数据点
        mi = compute_mutual_info(x[mask], y[mask])[feature_idx]
        return -mi  # 负值，因为 gp_minimize 是最小化函数
    return 0  # 如果数据点不足，返回 0

# 贝叶斯优化目标函数
def objective(bounds):
    return mutual_info_in_interval(bounds, X_feature, y_feature)

# 定义区间的搜索空间 (e.g., 第5维特征的取值在 （0，180） 之间)
space = [Real(0, 180, name='start'), Real(0, 180, name='end')]

# 使用贝叶斯优化寻找敏感性最高的区间
res = gp_minimize(objective, space, n_calls=20, random_state=0)

# 优化结果
best_bounds = res.x
print("Optimal Interval for x[:, 4]:", best_bounds)
print("Mutual Information in Optimal Interval:", -res.fun)

/home/home_node7/cjz/miniconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/home_node7/cjz/miniconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Optimal Interval for x[:, 4]: [0.0, 180.0]
Mutual Information in Optimal Interval: 0.06158931116192834


In [24]:
# 收集所有区间的结果并排序
results = sorted(
    [(bounds[0], bounds[1], -mi) for bounds, mi in zip(res.x_iters, res.func_vals)],
    key=lambda x: x[2],  # 以互信息值排序
    reverse=True
)

# 输出排名前三的敏感性区间及其敏感性
top_3_intervals = results[:3]
print("Top 3 Intervals with Highest Sensitivity (Mutual Information):")
for i, (start, end, mi) in enumerate(top_3_intervals, 1):
    print(f"Rank {i}: Interval [{start:.2f}, {end:.2f}], Mutual Information: {mi:.4f}")

Top 3 Intervals with Highest Sensitivity (Mutual Information):
Rank 1: Interval [0.00, 180.00], Mutual Information: 0.0616
Rank 2: Interval [106.71, 151.97], Mutual Information: -0.0000
Rank 3: Interval [154.43, 152.51], Mutual Information: -0.0000
